<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E6%9C%88%E7%B5%90Tpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gspread-dataframe

In [ ]:
pip install pygsheets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 1.1 MB/s eta 0:00:00


In [ ]:
import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import numpy as np
import pygsheets

## 存取「消費紀錄」

In [99]:
#HOME_PATH = '/home/iss/realplus/'
#HOME_PATH = '/home/iss/realplus/'
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1eHm_1rIPSQgUqYNStKq8kfMnGiQnfUDhxGEj1I15Fls/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("客戶交易明細表")
df = wks.get_as_df(start='A1', numerize=False )
df

/usr/local/lib/python3.10/dist-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


,,,,,,十里方圓,,,,,,
0,,,,,,客戶交易明細表,,,,,,
1,印表日期:,2023/11/01,,,,,,,,,頁次:,1
2,客戶區間:,..2427,,~,Y927659,,,報表種類:,明細表,,,
3,日期區間:,2023/10/01,~,2023/10/31,,,,分析單據:,銷貨單 + 銷貨退回單,,,
4,日期,單別,單據編號,商品編號,,商品名稱,,,,數量,單價,金額
...,...,...,...,...,...,...,...,...,...,...,...,...
300,,,,業務:,發票號碼:,TQ47966539,稅金:,0,,,,100
301,2023/10/31,銷貨,202310310006,S00802-0,,ST有善田園洛神黑麥汁,,1.00,,,50.00,50
302,,,,業務:,發票號碼:,TQ47966540,稅金:,0,,,,50
303,合計:,,,,,,稅金:,0,,,,150


In [100]:
from datetime import datetime
date_month = df.iloc[3, 1]
date_month
# 将字符串转换为 datetime 对象
date_month = datetime.strptime(date_month, '%Y/%m/%d')
# 提取年和月，并构建新的日期字符串
date_month = date_month.strftime('%Y-%m')
date_month

'2023-10'

In [101]:
df = wks.get_as_df(start='A6', numerize=False )
#wks.clear() #讀取資料後清空，避免重複執行，到了下一個cell因為沒有資料讀入，在執行iloc會報錯IndexError，這樣程式就可以順利終止了
# 新的列名列表
new_column_names = ['日期', '單別', '單據編號', '商品編號', 'Unnamed: 4', '商品名稱', 'Unnamed: 6',
                    'Unnamed: 7', 'Unnamed: 8', '數量', '單價', '金額']
# 更新 DataFrame 的列名
df.columns = new_column_names

# 將所有空字符串 ('') 替換為 NaN
df.replace('', np.nan, inplace=True)

df

/usr/local/lib/python3.10/dist-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


,日期,單別,單據編號,商品編號,Unnamed: 4,商品名稱,Unnamed: 6,Unnamed: 7,Unnamed: 8,數量,單價,金額
0,客戶編號:,000,NaN,客戶名稱:,NaN,NaN,NaN,NaN,NaN,統一編號:,NaN,NaN
1,2023/10/02,銷貨,202310020002,S00801-0,NaN,ST有善田園茂谷汽水,NaN,1.00,NaN,NaN,50.00,50
2,NaN,NaN,NaN,業務:,發票號碼:,TQ47966456,稅金:,0,NaN,NaN,NaN,50
3,2023/10/02,銷貨,202310020003,S00801-0,NaN,ST有善田園茂谷汽水,NaN,1.00,NaN,NaN,50.00,50
4,NaN,NaN,NaN,S00802-0,NaN,ST有善田園洛神黑麥汁,NaN,1.00,NaN,NaN,50.00,50
...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,業務:,發票號碼:,TQ47966539,稅金:,0,NaN,NaN,NaN,100
296,2023/10/31,銷貨,202310310006,S00802-0,NaN,ST有善田園洛神黑麥汁,NaN,1.00,NaN,NaN,50.00,50
297,NaN,NaN,NaN,業務:,發票號碼:,TQ47966540,稅金:,0,NaN,NaN,NaN,50
298,合計:,NaN,NaN,NaN,NaN,NaN,稅金:,0,NaN,NaN,NaN,150


## 比對「該月份消費紀錄」是否已經登記，如果登記則終止

In [102]:
# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1eHm_1rIPSQgUqYNStKq8kfMnGiQnfUDhxGEj1I15Fls/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
check = sht.worksheet_by_title("已讀取月份")
checklist = check.get_as_df(start='A1', numerize=False, header=False)
checklist.iloc[:,0]


0    2023-10
Name: 已讀資料（月份）, dtype: object

In [103]:
import sys
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 检查 date_month 是否在 checklist.iloc[:, 0] 中
if date_month not in checklist.iloc[:, 0].values:
    # 如果不在，则将其添加到最后一行
    last_row_index = len(checklist)
    checklist.loc[last_row_index] = [date_month] + [0] * (len(checklist.columns) - 1)

    ### 將checklist上傳（更新）到Google sheet儲存，以備下次查證。
    # Authenticate and open the spreadsheet
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('orderdata-copyto-parttime-5c19ba8fcba4.json', scope)
    client = gspread.authorize(creds)

    # Open the Google Sheet by URL
    sheet_id = "1eHm_1rIPSQgUqYNStKq8kfMnGiQnfUDhxGEj1I15Fls"
    sheet = client.open_by_key(sheet_id).get_worksheet(2)  # Open the third sheet

    # Convert DataFrame to a list of lists (including the header)
    data = [checklist.columns.values.tolist()] + checklist.values.tolist()

    # Update the sheet with the data
    sheet.update('A1', data)

else:# 如果已存在，则停止程序运行
    print(f"{date_month} already exists in the checklist. Exiting...")
    sys.exit()

2023-10 already exists in the checklist. Exiting...


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [96]:
# Define the column names and the value to search for based on the user's request
column_to_check = '日期'
value_to_search_for = '客戶編號:'
column_to_return = '單別'

# Find the rows where column_to_check equals value_to_search_for
matching_rows = df[df[column_to_check] == value_to_search_for]

# Get the values from column_to_return for these rows
matching_values = matching_rows[column_to_return].tolist()

# Return the matching row indices and their corresponding values in column_to_return
result_indices = matching_rows.index.tolist()
result = dict(zip(result_indices, matching_values))

import re
# Regular expression to match the pattern '1 uppercase letter followed by 6 digits'
pattern = r'^[A-Z]\d{6}$'

# Filtering the dictionary
result = {k: v for k, v in result.items() if re.match(pattern, v)}

print(result)
print(len(result))

{159: 'A862233', 165: 'B169595', 169: 'B231568', 173: 'D995235', 177: 'E789764', 181: 'H679207', 195: 'J033579', 199: 'J063816', 203: 'J200318', 218: 'J436617', 240: 'J692508', 244: 'J925577', 251: 'J933990', 260: 'M829221', 264: 'N221983', 276: 'N981930', 286: 'O951455', 292: 'R313833'}
18


In [97]:
index = min(list(result.keys()))  # starting index, adjust as necessary
a = []
# Loop until the end of the DataFrame
while index < len(df)-2:  # assuming df is your DataFrame and has at least 2 rows
    #print(index)
    # Check your conditions
    if df["Unnamed: 6"][index+2] == "稅金:" and df["Unnamed: 6"][index+1] == "稅金:" and pd.isnull(df["Unnamed: 6"][index]):
        amount = int(df['金額'][index+2].replace(',', ''))
        print(f"{index+2}:{amount}")  # Corrected the quotes inside the f-string
        a.append(amount)
    #else: print("NO")
    # Increment index at the end of each iteration
    index += 1
#print(a)

164:480
168:160
172:100
176:110
180:1200
194:1030
198:50
202:60
217:2298
239:7390
243:220
250:1220
259:1030
263:350
275:1950
285:819
291:160
298:150


In [42]:
# Checking if the lengths of result.values() and a are the same
len_result_values = len(result.values())
len_a = len(a)
lengths_match = len_result_values == len_a

# Creating the 'record' dictionary if lengths match
record = {}
if lengths_match:
    record = dict(zip(result.values(), a))

lengths_match

True

In [43]:
# 轉換為新的字典格式
transformed_record = {
    '社員編碼': list(record.keys()),
    '本月點數': list(record.values())
}

transformed_record


{'社員編碼': ['A862233',
  'B169595',
  'B231568',
  'D995235',
  'E789764',
  'H679207',
  'J033579',
  'J063816',
  'J200318',
  'J436617',
  'J692508',
  'J925577',
  'J933990',
  'M829221',
  'N221983',
  'N981930',
  'O951455',
  'R313833'],
 '本月點數': [480,
  160,
  100,
  110,
  1200,
  1030,
  50,
  60,
  2298,
  7390,
  220,
  1220,
  1030,
  350,
  1950,
  819,
  160,
  150]}

In [44]:
# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/19lfmsgopDkKvIw5TFqx1QptgVu4OOQz8EwC4L7SRQxk/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("修改")

df = wks.get_as_df()

# 將所有空字符串 ('') 替換為 0
df["本月點數"].replace('', 0, inplace=True)
df["累積點數"].replace('', 0, inplace=True)

df

,姓名,電話/手機號碼,電子郵件Email,社員編碼,本月點數,累積點數
0,林福仁,939271983,frlin@iss.nthu.edu.tw,N221983,1950,1950
1,蘇至弘,958137850,smbs202012@gmail.com,J147850,0,0
2,陳鼎元,953261568,dingyuan1022@yahoo.com,B231568,100,100
3,連文彬,918003725,a0918003725@gmail.com,K053725,0,0
4,蔡寬福,921841699,kentsai@makoto.com.tw,L691699,0,0
...,...,...,...,...,...,...
144,尹秀蓮,935151300,ailena0935151300@gmail.com,J511300,0,0
145,張淑嫺,908387236,sschang@mx.nthu.edu.tw,Q897236,0,0
146,王致凱,912881035,nper8203@gmail.com,F951035,0,0
147,李玉芬,965029007,yflee305@hotmail.com,T589007\r\n,0,0


In [47]:
# 測試時請確認原本的點數，避免重複計算
#df["本月點數"]=0
#df["累積點數"]=0

In [48]:
for id in transformed_record['社員編碼']:
    mask = df['社員編碼'] == id
    df.loc[mask, '本月點數'] = record.get(id, df.loc[mask, '本月點數'].values[0])
    current_month_points = df.loc[mask, '本月點數'].values[0]
    df.loc[mask, '累積點數'] += current_month_points

In [49]:
df

,姓名,電話/手機號碼,電子郵件Email,社員編碼,本月點數,累積點數
0,林福仁,939271983,frlin@iss.nthu.edu.tw,N221983,1950,1950
1,蘇至弘,958137850,smbs202012@gmail.com,J147850,0,0
2,陳鼎元,953261568,dingyuan1022@yahoo.com,B231568,100,100
3,連文彬,918003725,a0918003725@gmail.com,K053725,0,0
4,蔡寬福,921841699,kentsai@makoto.com.tw,L691699,0,0
...,...,...,...,...,...,...
144,尹秀蓮,935151300,ailena0935151300@gmail.com,J511300,0,0
145,張淑嫺,908387236,sschang@mx.nthu.edu.tw,Q897236,0,0
146,王致凱,912881035,nper8203@gmail.com,F951035,0,0
147,李玉芬,965029007,yflee305@hotmail.com,T589007\r\n,0,0


## 將資料覆寫進原本的資料。並寄給自己一份做為備份。

In [ ]:
pip install gspread oauth2client


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate and open the spreadsheet
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('orderdata-copyto-parttime-5c19ba8fcba4.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by URL
sheet_id = "19lfmsgopDkKvIw5TFqx1QptgVu4OOQz8EwC4L7SRQxk"
sheet = client.open_by_key(sheet_id).sheet1  # Open the first sheet

# Convert DataFrame to a list of lists (including the header)
data = [df.columns.values.tolist()] + df.values.tolist()

# Update the sheet with the data
sheet.update('A1', data)


{'spreadsheetId': '19lfmsgopDkKvIw5TFqx1QptgVu4OOQz8EwC4L7SRQxk',
 'updatedRange': "'修改'!A1:F150",
 'updatedRows': 150,
 'updatedColumns': 6,
 'updatedCells': 900}

In [ ]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from datetime import datetime

current_month = datetime.now().month

# Save DataFrame to Excel
excel_file = f'{date_month}月-Tpoint Excel File.xlsx'
df.to_excel(excel_file, index=False)

# Email configuration
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "jwsscmoiffdyukmm"
# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")

receiver_email = 'coop4s.hsinchu@gmail.com'

# Email content
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = f'{date_month}月: T-point Excel File'

# Attach the file to the email
with open(excel_file, 'rb') as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename= {excel_file}')
    message.attach(part)

# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.send_message(message)

# Delete the generated Excel file after sending the email
import os
os.remove(excel_file)

#準備寄信給「有點數的會員」進行點數告知
mail_df = df[df['累積點數'] != 0]

## 寄給大家一份

In [ ]:
'''
#測試
# 生成一組隨機資料
test_data = {
    "姓名": "陳皆錫",
    "電話/手機號碼": "0967252868",
    "電子郵件Email": "ubo11185@gapp.nthu.edu.tw",
    "社員編碼": "L88888",
    "本月點數": 100,
    "累積點數": 150
}
test_data1 = {
    "姓名": "吳博士(鋒測試)",
    "電話/手機號碼": "0912346578",
    "電子郵件Email": "1208541409@qq.com",
    "社員編碼": "A666666",
    "本月點數": 500,
    "累積點數": 500
}

# 定義欄位名稱
columns = ['姓名', '電話/手機號碼', '電子郵件Email', '社員編碼', '本月點數', '累積點數']

# 創建一個空的 DataFrame
testdf = pd.DataFrame(columns=columns)

#將資料加入 DataFrame
mail_df = testdf.append(test_data, ignore_index=True)
mail_df = mail_df.append(test_data1, ignore_index=True)
mail_df
'''

<ipython-input-36-726ae1309e8a>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mail_df = testdf.append(test_data, ignore_index=True)
<ipython-input-36-726ae1309e8a>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mail_df = mail_df.append(test_data1, ignore_index=True)


,姓名,電話/手機號碼,電子郵件Email,社員編碼,本月點數,累積點數
0,陳皆錫,0967252868,ubo11185@gapp.nthu.edu.tw,L88888,100,150
1,吳博士(鋒測試),0912346578,1208541409@qq.com,A666666,500,500


In [ ]:
mail_df

,姓名,電話/手機號碼,電子郵件Email,社員編碼,本月點數,累積點數
0,陳皆錫,0967252868,ubo11185@gapp.nthu.edu.tw,L88888,100,150
1,吳博士(鋒測試),0912346578,1208541409@qq.com,A666666,500,500


In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os


def email_sent(): #輸入的資料承接前面的cell是「mail_df」
  # 設定寄件者的電子郵件地址和密碼作為環境變數
  os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
  os.environ["COOP_PASSWORD"] = "coop4s@nthu"
  print("===================================")
        # Your loop for sending emails
  for index, row in mail_df.iterrows():
            recipient_name = row["姓名"]
            recipient_num = row["社員編碼"]
            amount = row["本月點數"]
            total = row["累積點數"]


            # Create email content
            email_content = f"合作社社員您好，本合作社通知您，您 {date_month}月 在「十里方圓合作社」進行的消費金額以及獲得的T-Point，詳細情況如下：\n\n"
            email_content += f"===================================\n"
            email_content += f"姓名：{recipient_name} \n"
            email_content += f"會員編號：：{recipient_num} \n"
            email_content += f"本月消費金額為: {amount} 元新台幣\n"
            email_content += f"本月消費獲得T-Point: {amount} 點數\n"
            email_content += f"本年度累計T-Point為: {total} 點數\n"
            email_content += f"===================================\n"
            email_content += f"感謝您對「十里方圓合作社」一直以來的支持，祝您以及家人健康平安，生活愉快。\n"
            email_content += f"===================================\n"

            # Email settings
            sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
            sender_password = "jwsscmoiffdyukmm"
            recipient_email = str(row["電子郵件Email"])  # 確保將電子郵件地址轉換為字符串

            subject = "【通知】十里方圓合作社社員「當月」消費額 &「年度」T-Point累積點數"
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = recipient_email
            message["Subject"] = subject
            message.attach(MIMEText(email_content, "plain"))

            try:
                # Connect to SMTP server
                server = smtplib.SMTP("smtp.gmail.com", 587)
                server.starttls()
                server.login(sender_email, sender_password)

                # Send the email
                server.sendmail(sender_email, recipient_email, message.as_string())

                # Close the SMTP server
                server.quit()

                print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
            except Exception as e:
                print(f"Failed to send email to {recipient_name}. Error: {str(e)}")

email_sent()

Email sent to 陳皆錫, recipient's email address: ubo11185@gapp.nthu.edu.tw
Email sent to 吳博士(鋒測試), recipient's email address: 1208541409@qq.com
